In [1]:
# In part one we obtained photo captions by scraping a socialite blog/website
# Here we extract the names of people from the captions

# This project will scrape a New York Social Diary website and identify divs with
# photos and photo captions. Parsing the captions will yield a social network graph 
# of the New York Social Elite.

In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
import requests
from bs4 import BeautifulSoup
import urllib2
import re
from xml.sax.saxutils import escape
from HTMLParser import HTMLParser
import numpy as np
from multiprocessing import Pool
import networkx as nx
import pickle
import dill
import time
import itertools
from itertools import chain
import csv
#import nltk
#from nltk.tag import pos_tag
import matplotlib.pyplot as plt

In [3]:
def findword(item):
    try:
        
            if (len(str(item))<250):
                
                item=str(item)
                item= item.replace(" von"," Von") # try  " von " and " van "
                item= item.replace(" van"," Van")
                item= item.replace(" de"," De") # changes Frederick to Fre Derick...
                sentense= str(re.findall(r'''([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)''',item))
                sentense = sentense.replace("Council ","")
                sentense = sentense.replace("Councilman ","")
                sentense = sentense.replace("Councilmember ","")
                sentense = sentense.replace("Councilwoman ","")
                sentense = sentense.replace("Congresswoman ","")
                sentense = sentense.replace("New York","")
                sentense = sentense.replace("Board","")
                sentense = sentense.replace("Member","")
                sentense = sentense.replace("Congressman ","")
                sentense = sentense.replace("Opening Gala","")
                sentense = sentense.replace("History","")
                sentense = sentense.replace("Museum","")
                sentense = sentense.replace("American","")
                sentense = sentense.replace("Congressman ","")
                sentense = sentense.replace("Night ","")
                sentense = sentense.replace("Miss ","")
                sentense = sentense.replace("USA ","")         
                sentense = sentense.replace("Mayor ","")
                sentense = sentense.replace("Event Co","") # ???
                sentense = sentense.replace("Count ","")
                sentense = sentense.replace("Countess ","")
                sentense = sentense.replace("Chairman ","")
                sentense = sentense.replace("Chairwoman ","")
                sentense = sentense.replace("President ","")
                sentense = sentense.replace("Museum ","")
                sentense = sentense.replace("Chairmen ","")
                sentense = sentense.replace("DJ ","")
                sentense = sentense.replace("MD ","")
                sentense = sentense.replace("Dance ","")
                sentense = sentense.replace("Director ","")
                sentense = sentense.replace("Dr. ","")
                sentense = sentense.replace("Sir ","")
                sentense = sentense.replace("The ","")
                sentense = sentense.replace("Society","")
                sentense = sentense.replace("Memorial Sloan","")                
                sentense = sentense.replace("Jr. ","")
                sentense = sentense.replace("CEO ","")
                sentense = sentense.replace("[", "")    # these ones ???
                sentense = sentense.replace("]", "")
                sentense = sentense.replace("' ", "'")
                sentense = sentense.replace(" ' ", "'")
                sentense = sentense.replace(" '", "'")
                sentense = sentense.replace("'", "")
                sentense = sentense.replace("Executive Director","")
                sentense = sentense.replace("Presi","") # not needed
                sentense = sentense.replace("Co ","")
                sentense = sentense.replace("Steering Committee ","")
                sentense = sentense.replace("Vice Presi ","") # not needed ?
                sentense = sentense.replace("Legal Defense","")
                sentense = sentense.replace("Educational Fund","")
                sentense = sentense.replace("National Equal Justice Award Dinner","")
                sentense = sentense.replace("Annual Gala","")
                #['Committee  Katie Boes']
                #['Dinosaur Fossil Table']
                #['Zoological Conservation Center']
                #['Magician Looney Louie']
                #['Science Center']
                #['Science Center']
                #['Three Little Operation Smile Bears']
                #['Brooklyn Food Coalition Kady Ferguson']
                #['Ross Award Winner David Schwarz']
                #['Ross Award Winner Edward Fraughton']
                #['Sister Jane Gerety']
                #['Holiday House Hamptons Designers']
                #['Frederick Law Olmsted Awards Luncheon']
                #['Principal Dancer Ashley Bouder']
                #['Soloist Justin Peck']
                #['Mount Sinai Crystal Party']
                #['Central Park Conservatory Garden']
                # errors to be added ... ['Speaker Melissa Mark']
                #['Family Dynamics Art Auction'] 
                #['Taittinger Nocturne Champagne']
                #['Friars Club Dean Emeritus Freddie Roman']
                return list(sentense.split(','))
           
            
    except Exception, f:
        y=f
        print "Hell broke loose in get nltp area!"+str(y)

In [4]:
list_of_words=[]    
f = open('NOV16_az_captions_full.txt', 'r') # test_az_captions.txt
for line in iter (f):
    w=findword(line)
    if str(w) == str(['']):
        pass
    else:
        list_of_words.append(w)
f.close()
list_of_words = [x for x in list_of_words if x != []]

In [5]:
print list_of_words[18]

['Bart Scott', 'Mark Laplander']


In [70]:
with open('new_filename_NOV16_full.txt', 'w') as f:
    for s in list_of_words:
        f.write(str(s) + '\n')
        
# save to pickle
with open("all_names_for_graph_full.pickle", "wb") as file:
    pickle.dump(list_of_words, file)

In [7]:
list_of_words[0:2]

[['Les Lieberman',
  'Barri Lieberman',
  'Isabel Kallman',
  'Trish Iervolino',
  'Ron Iervolino'],
 ['Chuck Grodin']]

In [12]:
# check for empty strings from regex ?
sumss = 0
for line in list_of_words:
    if str(line) == str(['']):
        #print line,str(line)
        sumss +=1
print sumss
#str(data[18]) == str([''])

0


In [64]:
data_combo=[]
data=[]
sumss  = 0

for line in list_of_words:
    if line is None:
        pass
    elif len(line) == 1:
        sumss += 1
    else:
        data.append(line)  
        #print line

print "there were "+ str(sumss) + " single names" # these are not needed and are dropped later in the code

there were 30674 single names


In [14]:
#data

In [65]:
with open('new_filename_NOV16_full_doubleormore.txt', 'w') as f:
    for s in data:
        f.write(str(s) + '\n')

In [15]:
len(data),len(list_of_words),len(data)-len(list_of_words)

(67685, 98361, -30676)

In [ ]:
# In part 3 we will form a social network graph and calculate best friends, page rank and most popular people
# In part 4 we explore visualization tools such as graph_tool and Gephi for social graph networks